In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

from __future__ import annotations
import os
from transitions import Machine
import threading
from typing import List
from queue import Queue
from type_protocols import StateActionObject, StatefulObject
import tcod
from PIL import Image
import numpy as np
import time
import queue

class GameLoopHandler:
    
    def handle(self, event: tcod.event.Event) -> None:
        """Handles the given event."""
        pass

class GameLoop:
    machine: Machine
    events: Queue[GameEvent | tcod.event.Event] | None
    actions: Queue[GameAction] | None
    handler: GameLoopHandler | None
    threads: List[threading.Thread | None]
    stop_signal: threading.Event

    def __init__(self) -> None:
        self.events = Queue()
        self.actions = Queue()
        self.handler = None
        self.threads = []
        self.stop_signal = threading.Event()
        threading.excepthook = self.threaded_exception_handler

        states = ['idle', 
                  {'name': 'started', 'on_enter': '_start'}, 
                  {'name': 'stopped', 'on_enter': '_stop'}]
        transitions =[
            {'trigger': 'start', 'source': 'stopped', 'dest': 'started'},
            {'trigger': 'stop', 'source': 'started', 'dest': 'stopped'}
            ]
        self.machine = Machine(model=self, states=states, transitions=transitions, initial='stopped')

        self.threads.append(threading.Thread(target=self.event_loop))
        self.threads.append(threading.Thread(target=self.action_loop))

    def _start(self) -> None:
        """Starts the game loop threads."""
        self.stop_signal.clear()
        for thread in self.threads:
            if thread is not None and not thread.is_alive():
                thread.start()
        print("Game loop has started.")

    def _stop(self) -> None:
        """Stops the game loop threads."""
        try:
            self.stop_signal.set()
            for thread in self.threads:
                if thread is not None and not thread.is_alive():
                    thread.join()
            print("Game loop has stopped.")

        except Exception as e:
            print(f"Error stopping game loop: {e}")
            
    def action_loop(self) -> None:
        """
        Update the state of the game by processing events and updating the roster, map, and UI.
        """
        while not self.stop_signal.is_set():
            try:
                next_action = None
                if self.actions is not None:
                    next_action = self.actions.get_nowait()
                if next_action is not None and isinstance(next_action, GameAction):
                    next_action.perform()
                
            except queue.Empty:
                time.sleep(0.05)

            except BaseException as e:
                print(f"Error processing action: {e}")
                break
    
    def event_loop(self) -> None:
        """
        Update the state of the game by processing events and updating the roster, map, and UI.
        """
        while not self.stop_signal.is_set():
            try:
                next_event = None
                if self.events is not None:
                    next_event = self.events.get_nowait()
                if next_event is not None and isinstance(next_event, GameEvent):
                    next_event.trigger()
                
            except queue.Empty:
                time.sleep(0.05)

            except BaseException as e:
                print(f"Error processing event: {e}")
                break

    def threaded_exception_handler(self, args):
        print(f"Thread failed: {args.thread.name}")
        print(f"Exception type: {args.exc_type}")
        print(f"Exception value: {args.exc_value}")
        print(f"Exception traceback: {args.exc_traceback}")

class GameStore:
    machine: Machine
    portfolio: Portfolio | None
    atlas: Atlas | None

    def __init__(self) -> None:
        self.portfolio = Portfolio()
        self.atlas = Atlas()
        self.display = Display()

        states = ['idle', 
                  {'name': 'started', 'on_enter': '_start'}, 
                  {'name': 'stopped', 'on_enter': '_stop'}]
        transitions =[
            {'trigger': 'start', 'source': 'stopped', 'dest': 'started'},
            {'trigger': 'stop', 'source': 'started', 'dest': 'stopped'}
            ]
        self.machine = Machine(model=self, states=states, transitions=transitions, initial='stopped')


    def _start(self):
        """Starts the game loop and prepares the game state for play."""
        # self.atlas.create_map()
        # self.map = self.atlas.active

        # if self.map is not None:
        #     self.library.spawn_player(self.map)
        #     self.library.initialize_random_mobs(self.map, max_mobs_per_area=3)
        #     self.player = self.library.player
        #     if self.player is not None:
        #         self.player.fov_radius = 6
        #     self.mobs = self.library.live_ai_actors
        
        # # self.engage_loop()
        print("Game Store has started.")

    def _stop(self):
        """Handles any cleanup or finalization needed when the game stops."""
        print("Game Store has stopped.")
        # self.portfolio = None
        # self.atlas = None
        # self.loop.stop.set() # Confirm all threads terminate.
    
    def enqueue_event(self, event: tcod.event.Event) -> None:
        """Publishes the given event to the main loop's event queue."""
        # if self.loop and self.loop.events is not None:
        #     self.loop.events.put(event)
        pass

class GameEvent:
    def trigger(self) -> None:
        """Triggers the game event."""
        pass

class GameAction:
    def perform(self) -> None:
        """Performs the game action."""
        pass

class Display:
    machine: Machine
    context: tcod.context.Context
    log: StatefulObject

    TITLE = "JRL - Jay's Roguelike"
    WIDTH, HEIGHT = 200, 96  # Window pixel resolution (when not maximized.)
    FLAGS = tcod.context.SDL_WINDOW_RESIZABLE | tcod.context.SDL_WINDOW_MAXIMIZED
    TILESET = tcod.tileset.load_truetype_font(os.path.join("src","core_components", "ui", "graphics", "resources", "GoogleSansCode-SemiBold.ttf"), 25, 25)
    
    def __init__(self) -> None:
        states = ['idle', 
                    {'name': 'started', 'on_enter': '_start'}, 
                    {'name': 'stopped', 'on_enter': '_stop'}]
        transitions =[
            {'trigger': 'start', 'source': 'stopped', 'dest': 'started'},
            {'trigger': 'stop', 'source': 'started', 'dest': 'stopped'}
            ]
        self.machine = Machine(model=self, states=states, transitions=transitions, initial='stopped')

        img = Image.open(os.path.join("src","core_components", "ui", "graphics", "resources", "player", "test-5.png"))
        img = img.convert("RGBA")
        self.TILESET.set_tile(64, np.array(img))
        img = Image.open(os.path.join("src","core_components", "ui", "graphics", "resources", "mob", "test-3.png"))
        img = img.convert("RGBA")
        self.TILESET.set_tile(65, np.array(img))

    def _start(self) -> None:
        """Initializes the display for rendering."""
        self.context = tcod.context.new(columns = self.WIDTH, rows = self.HEIGHT, tileset=self.TILESET, title=self.TITLE, vsync=True, sdl_window_flags=self.FLAGS)

        print("Display has started.")

    def _stop(self) -> None:
        """Cleans up resources used by the display."""
        self.context.close()
        print("Display has stopped.")

    def render(self) -> None:
        """Renders the current game state to the display."""
        pass
    
class Portfolio:
    machine: Machine
    log: StatefulObject
    def enqueue_events(self) -> None:
        """Publishes any events related to the portfolio."""
        pass

class Atlas:
    machine: Machine

    def enqueue_events(self) -> None:
        """Publishes any events related to the atlas."""
        pass

class GameEngine:
    """
    The Game updates the game state in the main loop. It is has States that it passes to the Game AI. 
    The GameAI converts states to a sequence of actions that the Game performs in the main game loop.

    Duck Types: StatefulObject, StateReferenceObject
    """
    machine: Machine
    loop: GameLoop | None
    store: GameStore | None
    display: Display | None

    def __init__(self) -> None:
        self.loop = GameLoop()
        self.display = Display()
        self.store = GameStore()

        states = [{'name': 'idle', 'on_enter': '_initialize'}, 
                  {'name': 'playing', 'on_enter': '_play'},
                  {'name': 'paused', 'on_enter': '_pause'},
                  {'name': 'resuming', 'on_enter': '_play'},
                  {'name': 'shutdown', 'on_enter': '_shutdown'},
                  {'name': 'resetting', 'on_enter': '_reset'}]
        transitions =[
            {'trigger': 'start', 'source': 'shutdown', 'dest': 'idle'},
            {'trigger': 'play', 'source': 'idle', 'dest': 'playing'},
            {'trigger': 'pause', 'source': 'playing', 'dest': 'paused'},
            {'trigger': 'resume', 'source': 'paused', 'dest': 'resuming'},
            {'trigger': 'stop', 'source': 'idle', 'dest': 'shutdown'},
            {'trigger': 'stop', 'source': 'playing', 'dest': 'shutdown'},
            {'trigger': 'reset', 'source': 'shutdown', 'dest': 'resetting'},
            {'trigger': 'reset', 'source': 'playing', 'dest': 'resetting'},
            {'trigger': 'reset', 'source': 'idle', 'dest': 'resetting'},
            {'trigger': 'restart', 'source': 'resetting', 'dest': 'idle'},
            ]
        self.machine = Machine(model=self, states=states, transitions=transitions, initial='shutdown')


        #self.library.log.add("Welcome to Jay's Roguelike!")

    def _initialize(self) -> None:
        """Initializes the game state, including the portfolio, atlas, and display."""
        print("Initializing the game.")
        if self.display:
            self.display.start() # type: ignore
        if self.loop:
            self.loop.start() # type: ignore

        print(f"Game is {self.state}.") # type: ignore
    
    def _play(self) -> None:
        """Starts the main game loop, processing events and updating the game state."""
        print("Starting the game.")
        if self.store:
            self.store.start() # type: ignore
    
        print(f"Game is {self.state}.") # type: ignore

    def _pause(self) -> None:
        """Pauses the game loop, halting event processing and state updates."""
        print("Pausing the game.")
        if self.store:
            self.store.stop() # type: ignore

        print(f"Game is {self.state}.") # type: ignore

    def _shutdown(self) -> None:
        """Cleans up resources and stops the game loop."""
        print("Shutting down the game.")
        if self.loop:
            self.loop.stop() # type: ignore
        if self.display:
            self.display.stop() # type: ignore

        print(f"Game is {self.state}.") # type: ignore

    def _reset(self) -> None:
        """Resets the game state to its initial configuration."""
        print("Resetting the game.")
        self.store = GameStore()
        self.restart() # type: ignore
        self.play() # type: ignore
        
        print(f"Game is {self.state}.") # type: ignore


In [4]:
def main() -> None:
    game = GameEngine()
    game.start() # type: ignore
    i = 1
    while True:
        # # Post Input Events
        # for event in tcod.event.wait():
        #     game.enqueue_event(event)

        # # Post Element Events
        # if game.portfolio:
        #     game.portfolio.enqueue_events()
            
        # if game.atlas:
        #     game.atlas.enqueue_events()

        # Update Display
        if game.display:
            game.display.render()

        if i > 1000:
            if game.state == 'idle':   # type: ignore
                game.stop() # type: ignore
            
            if game.state == 'shutdown':   # type: ignore
                break
            
        i += 1

if __name__ == "__main__":
    main()

RENDER:Created renderer: direct3d11


Initializing the game.
Display has started.
Game loop has started.
Game is idle.
Shutting down the game.
Game loop has stopped.
Display has stopped.
Game is shutdown.
